# Chapter 11. Word2Vec, ELMo, Doc2Vec의 이해
* 단어를 원핫 벡터로 표현하는 것은 기계적이지만, 이 원핫 벡터를 밀집 벡터로 변환하려면 반드시 학습이 있어야 함 
* Word2Vec 기법에서는 학습의 목적
  * 단어에 의미적인 정보를 함축
  * 유사도를 계산
  * 연산을 수행하고
  * 학습된 결과를 다른 작업에서도 사용할 수 있는 전이학습을     지원하는 것 

## 11.1 Word2Vec- 대표적인 워드 임베딩 기법 
Word2Vec 학습 방법 
* 다음 단어 혹은 주변 단어에 대한 예측을 잘할 수 있도록 학습함으로써 문맥을 이해시키고, 밀집 벡터에 그러한 문맥 정보를 담으려는 시도 
  * CBOW(Continuout Bag of Words)
    * 주변의 단어를 이용해 중심에 있는 단어를 예측하도록 학습
    * window(윈도우): 앞뒤 단어들을 몇 개씩 예측에 사용할지 결정하는 범위
    * 단어에 대한 원핫 벡터를 밀집 벡터로 변환하는 가중치 행렬 자체가 변환된 밀집 벡터를 표시하고 있음  

  * Skip-Gram
    * 중심의 한 단어를 이용해 주변의 단어 예측 

In [ ]:
#------------------------------------------------
# 11.1.2 Word2Vec 활용 - 학습된 모형 가져오기
#------------------------------------------------
# 대상 말뭉치에 대해 직접 학습을 함으로써 나만의 가중치 행렬 혹은 워드 임베딩 벡터를 생성하는 것 
# 기존에 학습된 임베딩 벡터를 가져다 활용하는 방법 
# Word2Vec 학습 지원 라이브러리 : Gensim 
pip install --upgrade gensim

  Using cached gensim-4.1.2-cp38-cp38-win_amd64.whl (24.0 MB)
  Using cached Cython-0.29.23-cp38-cp38-win_amd64.whl (1.7 MB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 미리 학습된 gensim data를 다운로드
# 사용할 수 있는 사전학습된 모델은 gensim 홈페이지 참조 
import gensim.downloader as api

# 모델 이름 마지막 숫자: 임베딩 벡터의 크기 (여기선 50)
wv = api.load('glove-wiki-gigaword-50')

In [ ]:
# 1. 단어
# 2. 그 단어의 임베딩 벡터 간의 매핑을 저장한 사전 
# 3. 다양한 유사도 함수들로 구성된 객체 포함  
type(wv)

gensim.models.keyedvectors.KeyedVectors

In [ ]:
vec_king = wv['king'] # king의 임베딩 벡터 크기와 내용 출력 
print('#Size of the vector:', len(vec_king))
print('#Vector for king:', vec_king)

#Size of the vector: 50
#Vector for king: [ 0.50451   0.68607  -0.59517  -0.022801  0.60046  -0.13498  -0.08813
  0.47377  -0.61798  -0.31012  -0.076666  1.493    -0.034189 -0.98173
  0.68229   0.81722  -0.51874  -0.31503  -0.55809   0.66421   0.1961
 -0.13495  -0.11476  -0.30344   0.41177  -2.223    -1.0756   -1.0783
 -0.34354   0.33505   1.9927   -0.04234  -0.64319   0.71125   0.49159
  0.16754   0.34344  -0.25663  -0.8523    0.1661    0.40102   1.1685
 -1.0137   -0.21585  -0.15155   0.78321  -0.91241  -1.6106   -0.64426
 -0.51042 ]


In [ ]:
# wv 다양한 유사도 관련 함수 제공 
  # similarity
    # 두 단어 간의 거리를 계산 (cosine similarity)
    # wv.similarity('word','word')
  # most_similar
    # 주어진 단어 리스트와 가장 유사하거나 가장 거리가 먼 단어들을 유사도와 함께 구함 
    # wv.most_similart(positive =['word1','word2'], topn = 3)
    # wv.most_similar(positive= ['word1','word2'], negative = ['word3'], topn= 1)
  # doesnt_match
    # 주어진 단어 집합 중에서 다른 단어들과 가장 거리가 먼 단어를 골라냄 
    # wv.doesnt_match('breakfast cereal dinner lunch')
print(wv.similarity('king', 'man'), 'vs', wv.similarity('king', 'woman'))
print(wv.similarity('queen', 'man'), 'vs', wv.similarity('queen', 'woman'))
print('미니밴에 가까운 차:', wv.most_similar(positive=['car', 'minivan'], topn=3))
print('여성, 왕에는 가까우면서 남성과는 먼 단어:', 
      wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))
print('breakfast cereal dinner lunch 중에서 다른 단어들과의 거리가 가장 먼 단어:', 
      wv.doesnt_match("breakfast cereal dinner lunch".split()))

0.53093773 vs 0.41133785
0.53667 vs 0.60031056
미니밴에 가까운 차: [('truck', 0.9100274443626404), ('suv', 0.904007613658905), ('jeep', 0.8619830012321472)]
여성, 왕에는 가까우면서 남성과는 먼 단어: [('queen', 0.8523604273796082)]
breakfast cereal dinner lunch 중에서 다른 단어들과의 거리가 가장 먼 단어: cereal


In [ ]:
# wv 다양한 유사도 관련 함수 제공 
  # distance
    # 유사도와 반대 개념인 거리를 반환
    # wv.distance('cat','dog')
  # n_similarity 
    # 단어집합간의 유사도를 계산
    # wv.n_similarity(['word1','word2'], ['word3','word4'])
print("distance between cat and dog: {:.2f}".format(wv.distance("cat", "dog")))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['japanese', 'restaurant'])))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['korean', 'restaurant'])))
print("{:.4f}".format(wv.n_similarity(['bulgogi', 'shop'], ['french', 'restaurant'])))

distance between cat and dog: 0.08
0.5375
0.5627
0.4377


In [1]:
# Word2Vec은 전이학습 또는 트랜스퍼 러닝으로 학습시간 단축, 학습결과 향상하는 도구로 사용 
# 최근, BERT(Bidirectional Encoder Representations from Transformers) 사용 

In [ ]:
#------------------------------------------------
# 11.1.3 FastText - 워드 임베딩에서 N-gram 적용
#------------------------------------------------
# Word2Vec 의 단점 보완하는 워드 임베딩 방법 
  # GloVe(Global Vectors for Word Representation)
    # Word2Vec이 주변단어를 중심으로 학습되어 문서 전체에서 다른 단어들과의 관계가 반영되지 않는 점을 극복 
  # FastText
    # OOV(Out of Vocabulary),학습 문세어 없는 단어에 취약한 문제 해결 
    # 단어에 문자 단위를 N-gram을 적용함으로써 해결 
    # 단어의 부분단어(sub-word)를 토큰으로 사용함으로써 OOV, 즉 모르는 단어에 대한 유사도 계싼 가능 
    # 기존의 단어를 변형하거나 결합해서 만든 새로운 단어에 대해 뛰어난 유사도 

## 11.2 ELMO - 문맥에 따른 단어 의미의  구분

In [ ]:
#------------------------------------------------
# 11.2.1 Word2Vec의 문제점 
#------------------------------------------------
# 큰 문제점: 동음이의어
# 같은 배 - 다른 의미 (가슴 밑 배, 먹는 배)-> 같은 벡터에 할당됨 
# -> ELMo(Embeddings from Language Model), 문맥을 반영한 워드 임베딩 모형 
# Word2Vec 임베딩 벡터는 고정돼 있지만, ELMo는 가변적 
  # Word2Vec: 학습을 통해 생성한 고정된 임베딩 벡터를 가져다 씀 
  # ELMo: 학습된 모형을 가져와서 주어진 문장에 맞게 가변적인 임베딩 벡터 생성 

In [ ]:
#------------------------------------------------
# 11.2.2 ELMo의 구조 
#------------------------------------------------
# ELMo 는 언어모델(Language Model)을 이용해 임베딩을 수행
# 언어 모델
  # 문장 혹은 단어의 시퀀스에 대해 확률을 할당함으로써 그 문장이 얼마나 자연스러운지를 알 수 있음 
  # 앞에 나온 단어들을 이용해 다음 단어를 예측하는 모형 
  # 문맥에 대한 이해를 높임 

# ELMo
  # breakfast cereal dinner lunch
  # 사전학습된 양방향 LSTM(bi-LSTM)을 이용해 임베딩 수행
  # 양방향 LSTM은 앞 단어들이 뒤에 미치는 영향에 더해
  # 뒤 단어에서 앞 단어 방향으로도 LSTM 층을 추가해 반대 방향의 영향도 학습함 

# ELMo 의 구조 
  # 두 개의 양방향 LSTM 층으로 이루어짐 
  # 문장의 단어들을 먼저 일반적인 임베딩을 통해벡터로 변환 
  # 이 임베딩 벡터가 첫째 양방향 LSTM 층의 입력이 됨 
  # 동일한 임베딩 벡터가 정방향 LSTm 과 역방향 LSTM에 입력으로 들어가 각각의 출력을 만들어 냄 
  # 이 둘이 합쳐져 첫째 층의 출력이 됨 
  # 정방향, 역방향이 서로 합쳐지지 않고 각각 입력으로 사용됨 
  # 결과적으로 정방향 LSTM, 역방향 LSTM 이 독립적으로 학습됨 

## 11.3 Doc2Vec - 문맥을 고려한 문서 임베딩
* Doc2Vec은 문서에 대해 직접 임베딩 
* 문서의 ID를 단어와 동일하게 취급해서 학습과정에 포함 
* 이 문서 ID는 그 문서에서 나온 학습 입력집합에 모두 포함됨 
* Doc2Vec 의 두가지 학습 구조 
  * DM(Distributed Memory)
    * Word2Vec의 CBOW에 문서 ID를 추가한 형태의 학습
    * 앞의 단어들과 문서 ID를 이용해 다음 단어를 예측하는 방식
  * DBOW(Distributed Bag of Words)
    * Skip-Gram에 문서 ID를 추가한 형태 
    * 문서 ID로 일련의 단어들을 예측하는 방식
    
